In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import ast
import numpy as np

In [2]:
# --- CONFIGURATION ---
RESULTS_FILE = "results_perturb.csv"
HISTORY_DIR = "tests"
PLOTS_DIR = "plots"

# Create main plots directory
if not os.path.exists(PLOTS_DIR):
    os.makedirs(PLOTS_DIR)

# --- LOAD SUMMARY DATA ---
# This contains the Baseline and the Path string needed for the plots
if os.path.exists(os.path.join(HISTORY_DIR, RESULTS_FILE)):
    result_path = os.path.join(HISTORY_DIR, RESULTS_FILE)
else:
    result_path = RESULTS_FILE

summary_df = pd.read_csv(result_path)
print(f"Loaded summary results: {len(summary_df)} runs found.")

Loaded summary results: 24 runs found.


In [3]:
# HELPER FUNCTION
def get_matching_row(n, d, a, b, df):
    """
    Finds the row in the summary dataframe that matches the parameters.
    Handles float tolerance for density.
    """
    subset = df[
        (df['N'] == int(n)) & 
        (df['alpha'] == float(a)) & 
        (df['beta'] == float(b))
    ]
    
    if not subset.empty:
        match = subset[np.isclose(subset['density'], float(d), atol=1e-5)]
        if not match.empty:
            return match.iloc[0]
    return None

In [4]:
def plot_single_analysis():
    print("\n>>> Scanning 'tests' folder and generating plots...")

    # Iterate over every file in the tests directory
    for filename in os.listdir(HISTORY_DIR):
        if not filename.startswith("prob_") or not filename.endswith(".csv"):
            continue
            
        # Skip the summary file itself if it starts with prob (unlikely but safe)
        if filename == "prob_results.csv": 
            continue

        # 1. Parse Filename
        try:
            clean_name = filename.replace('.csv', '')
            parts = clean_name.split('_')
            
            if len(parts) != 5:
                print(f"Skipping {filename}: Unexpected format (expected prob_N_d_a_b)")
                continue
                
            _, n_str, d_str, a_str, b_str = parts
            
            # 2 - find Matching Summary Data
            row = get_matching_row(n_str, d_str, a_str, b_str, summary_df)
            
            if row is None:
                print(f"Skipping {filename}: No matching run found in {RESULTS_FILE}")
                continue
                
            # 3 - Setup Output Directory
            prob_dir_name = f"prob_{int(row['N'])}_{row['density']}_{row['alpha']}_{row['beta']}"
            prob_path = os.path.join(PLOTS_DIR, prob_dir_name)
            if not os.path.exists(prob_path):
                os.makedirs(prob_path)
                
            # 4 - Load History Data
            hist_df = pd.read_csv(os.path.join(HISTORY_DIR, filename))


            # PLOT A: Learning Process (Convergence)
            plt.figure(figsize=(10, 6))
            
            plt.plot(hist_df['Generation'], hist_df['Avg_Cost'], label='Average Population Cost', color='orange', alpha=0.5, linewidth=1.5)
            plt.plot(hist_df['Generation'], hist_df['Best_Cost'], label='Best Solution Cost', color='#1f77b4', linewidth=2.5)
            plt.axhline(y=row['Baseline'], color='#d62728', linestyle='--', linewidth=2, label='Baseline (Greedy)')
            
            if 'Stagnation_Count' in hist_df.columns:
                perturbations = hist_df[hist_df['Stagnation_Count'].diff() < -40]
                for gen in perturbations['Generation']:
                    plt.axvline(x=gen, color='purple', linestyle=':', alpha=0.6, label='Perturbation Triggered' if gen == perturbations['Generation'].iloc[0] else "")
            
            plt.title(f"Learning Process: N={int(row['N'])}, d={row['density']}, α={row['alpha']}, β={row['beta']}", fontsize=14)
            plt.xlabel("Generation")
            plt.ylabel("Total Cost")
            plt.legend()
            plt.tight_layout()
            plt.savefig(os.path.join(prob_path, "A_convergence.png"), dpi=300)
            plt.close()


            # PLOT B: Thief's Burden (Weight Profile)
            try:
                path_data = ast.literal_eval(row['Path'])
                
                weights = []
                current_w = 0.0
                steps = []
                
                for i, (node, gold) in enumerate(path_data):
                    steps.append(i)
                    if node == 0:
                        current_w = 0.0
                    else:
                        current_w += gold
                    weights.append(current_w)
                    
                plt.figure(figsize=(10, 5))
                
                plt.fill_between(steps, weights, color='#2ca02c', alpha=0.3)
                plt.plot(steps, weights, color='#2ca02c', linewidth=1.5)
                
                plt.title(f"Thief's Burden (Weight Profile)\nSteps: {len(steps)} | Final Cost: {row['Cost']:,.0f}", fontsize=14)
                plt.xlabel("Step Number (Physical Path)")
                plt.ylabel("Current Weight Carried")
                plt.grid(True, linestyle=':', alpha=0.6)
                
                plt.savefig(os.path.join(prob_path, "B_weight_profile.png"), dpi=300)
                plt.close()
                
            except Exception as e:
                print(f"Error generating Plot B for {filename}: {e}")
                
            print(f"Processed: {filename}")

        except Exception as e:
            print(f"Error processing {filename}: {e}")

    print("\nProcessing complete.")

In [5]:
def plot_comparative_analysis():
    # PLOT C: Scorecard (Improvement Grouped by Beta)
    print("Generating Plot C (Improvement by Beta)...")

    plt.figure(figsize=(10, 6))

    sns.boxplot(data=summary_df, x='beta', y='Improvement', palette='coolwarm')
    sns.stripplot(data=summary_df, x='beta', y='Improvement', color='black', alpha=0.5, jitter=True)

    plt.title("Scorecard: Algorithm Improvement vs. Baseline (Grouped by Beta)", fontsize=15)
    plt.xlabel("Beta (Weight Penalty Factor)", fontsize=12)
    plt.ylabel("Improvement over Baseline (%)", fontsize=12)
    plt.grid(axis='y', linestyle='--', alpha=0.7)

    plt.tight_layout()
    plt.savefig(os.path.join(PLOTS_DIR, "C_improvement_by_beta_boxplot.png"), dpi=300)
    plt.close()


    # PLOT D: Scalability (Time Analysis)
    print("Generating Plot D (Scalability/Time)...")

    # Aggregate by N to get average times
    time_df = summary_df.groupby('N')[['Time_Pre_Algo', 'Time_Algo']].mean().reset_index()

    plt.figure(figsize=(10, 6))

    p1 = plt.bar(time_df['N'].astype(str), time_df['Time_Pre_Algo'], color='#ff7f0e', label='Initialization Time (Pre-Algo)')
    p2 = plt.bar(time_df['N'].astype(str), time_df['Time_Algo'], bottom=time_df['Time_Pre_Algo'], color='#1f77b4', label='Evolution Time (Algo)')

    plt.title("Computational Time Analysis (Scalability)", fontsize=15)
    plt.xlabel("Number of Cities (N)", fontsize=12)
    plt.ylabel("Average Time (seconds)", fontsize=12)
    plt.legend()
    plt.grid(axis='y', linestyle='--', alpha=0.5)

    plt.tight_layout()
    plt.savefig(os.path.join(PLOTS_DIR, "D_scalability.png"), dpi=300)
    plt.close()


    # PLOT E: Parameter Sensitivity (Beta Trend)
    print("Generating Plot E (Beta Sensitivity)...")

    beta_trend = summary_df.groupby('beta')['Improvement'].mean().reset_index()

    plt.figure(figsize=(8, 5))

    plt.plot(beta_trend['beta'], beta_trend['Improvement'], marker='o', linestyle='-', color='purple', linewidth=2, markersize=8)

    for i, txt in enumerate(beta_trend['Improvement']):
        plt.annotate(f"{txt:.1f}%", (beta_trend['beta'][i], beta_trend['Improvement'][i]), 
                    textcoords="offset points", xytext=(0,10), ha='center', fontsize=11, fontweight='bold')

    plt.title("Parameter Sensitivity: Impact of Beta on Optimization Potential", fontsize=14)
    plt.xlabel("Beta Parameter", fontsize=12)
    plt.ylabel("Average Improvement (%)", fontsize=12)
    plt.grid(True, linestyle='--')
    plt.xticks(beta_trend['beta'])

    plt.tight_layout()
    plt.savefig(os.path.join(PLOTS_DIR, "E_beta_sensitivity.png"), dpi=300)
    plt.close()

    print(f"Comparative plots (C, D, E) saved to '{PLOTS_DIR}/'.")

In [6]:
plot_single_analysis()


>>> Scanning 'tests' folder and generating plots...
Processed: prob_1000_0.2_1_0.5.csv
Processed: prob_1000_0.2_1_1.csv
Processed: prob_1000_0.2_1_2.csv
Processed: prob_1000_0.2_2_1.csv
Processed: prob_1000_1.0_1_0.5.csv
Processed: prob_1000_1.0_1_1.csv
Processed: prob_1000_1.0_1_2.csv
Processed: prob_1000_1.0_2_1.csv
Processed: prob_100_0.2_1_0.5.csv
Processed: prob_100_0.2_1_1.csv
Processed: prob_100_0.2_1_2.csv
Processed: prob_100_0.2_2_1.csv
Processed: prob_100_1.0_1_0.5.csv
Processed: prob_100_1.0_1_1.csv
Processed: prob_100_1.0_1_2.csv
Processed: prob_100_1.0_2_1.csv
Processed: prob_400_0.2_1_0.5.csv
Processed: prob_400_0.2_1_1.csv
Processed: prob_400_0.2_1_2.csv
Processed: prob_400_0.2_2_1.csv
Processed: prob_400_1.0_1_0.5.csv
Processed: prob_400_1.0_1_1.csv
Processed: prob_400_1.0_1_2.csv
Processed: prob_400_1.0_2_1.csv

Processing complete.


In [7]:
plot_comparative_analysis()

Generating Plot C (Improvement by Beta)...


/tmp/ipykernel_267570/2922930303.py:7: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(data=summary_df, x='beta', y='Improvement', palette='coolwarm')


Generating Plot D (Scalability/Time)...
Generating Plot E (Beta Sensitivity)...
Comparative plots (C, D, E) saved to 'plots/'.
